In [1]:
import numpy as np
import pandas as pd

In [48]:
allMovies = pd.read_csv('../../Data/serendipity-sac2018/movies.csv', 
                     error_bad_lines=False)
imdbData = pd.read_csv('../Intermediate_data/IMDb_data.csv')
trainClusters = pd.read_csv('../Intermediate_data/train_clusters.csv')

Skipping line 19833: expected 8 fields, saw 10
Skipping line 34143: expected 8 fields, saw 9
Skipping line 36015: expected 8 fields, saw 10
Skipping line 37260: expected 8 fields, saw 12
Skipping line 44379: expected 8 fields, saw 10
Skipping line 47551: expected 8 fields, saw 10



In [3]:
trainClusters.head()

,Unnamed: 0,index,userId,movieId,rating,timestamp,year,title,releaseDate,directedBy,...,IMAX,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western,nan,cluster
0,0,25353,142748,2762,5.0,2017-12-31 23:59:41,2017,"Sixth Sense, The (1999)",1999-08-06,M. Night Shyamalan,...,0,0,1,0,0,0,0,0,0,0
1,1,26123,142752,2762,4.0,2017-12-31 20:41:29,2017,"Sixth Sense, The (1999)",1999-08-06,M. Night Shyamalan,...,0,0,1,0,0,0,0,0,0,3
2,2,26180,142744,2762,4.0,2017-12-31 20:35:25,2017,"Sixth Sense, The (1999)",1999-08-06,M. Night Shyamalan,...,0,0,1,0,0,0,0,0,0,1
3,3,26257,142753,2762,4.5,2017-12-31 20:19:35,2017,"Sixth Sense, The (1999)",1999-08-06,M. Night Shyamalan,...,0,0,1,0,0,0,0,0,0,0
4,4,26651,142747,2762,3.0,2017-12-31 18:02:45,2017,"Sixth Sense, The (1999)",1999-08-06,M. Night Shyamalan,...,0,0,1,0,0,0,0,0,0,0


In [6]:
imdbData.head()

,Unnamed: 0,certificates,country codes,genres,kind,language codes,plot,rating,runtimes,votes,year,imdbId
0,0,"[u'Argentina:Atp', u'Australia:G', u'Belgium:K...",[u'us'],"[u'Animation', u'Adventure', u'Comedy', u'Fami...",movie,[u'en'],"[u""A cowboy doll is profoundly threatened and ...",8.3,[u'81'],837115.0,1995.0,114709
1,1,"[u'Argentina:Atp', u'Australia:PG', u'Brazil:1...",[u'us'],"[u'Adventure', u'Comedy', u'Family', u'Fantasy']",movie,"[u'en', u'fr']",[u'When two kids find and play a magical board...,7.0,[u'104'],287007.0,1995.0,113497
2,2,"[u'Argentina:Atp', u'Australia:PG', u'Finland:...",[u'us'],"[u'Comedy', u'Romance']",movie,"[u'en', u'it', u'de']","[u""John and Max resolve to save their beloved ...",6.7,[u'101'],23107.0,1995.0,113228
3,3,"[u'Argentina:16', u'Australia:M', u'Canada:14A...",[u'us'],"[u'Comedy', u'Drama', u'Romance']",movie,[u'en'],"[u""Based on Terry McMillan's novel, this film ...",5.9,[u'124'],9034.0,1995.0,114885
4,4,"[u'Argentina:Atp', u'Australia:G', u'Finland:S...",[u'us'],"[u'Comedy', u'Family', u'Romance']",movie,[u'en'],[u'George Banks must deal not only with the pr...,6.0,[u'106'],32568.0,1995.0,113041


## Format IMDb features

In [49]:
imdbData = imdbData.dropna()

In [57]:
imdbData['country codes'] = imdbData['country codes'].apply(lambda x: [str(s) for s in ast.literal_eval(x)])
imdbData['language codes'] = imdbData['language codes'].apply(lambda x: [str(s) for s in ast.literal_eval(x)])
imdbData['genres'] = imdbData['genres'].apply(lambda x: [str(s) for s in ast.literal_eval(x)])
imdbData['runtimes'] = imdbData['runtimes'].apply(lambda x: [int(s) for s in ast.literal_eval(x)])

In [61]:
imdbData.drop('Unnamed: 0', inplace=True,axis=1)
imdbData.head()

,certificates,country codes,genres,kind,language codes,plot,rating,runtimes,votes,year,imdbId
0,"[u'Argentina:Atp', u'Australia:G', u'Belgium:K...",[us],"[Animation, Adventure, Comedy, Family, Fantasy]",movie,[en],"[u""A cowboy doll is profoundly threatened and ...",8.3,[81],837115.0,1995.0,114709
1,"[u'Argentina:Atp', u'Australia:PG', u'Brazil:1...",[us],"[Adventure, Comedy, Family, Fantasy]",movie,"[en, fr]",[u'When two kids find and play a magical board...,7.0,[104],287007.0,1995.0,113497
2,"[u'Argentina:Atp', u'Australia:PG', u'Finland:...",[us],"[Comedy, Romance]",movie,"[en, it, de]","[u""John and Max resolve to save their beloved ...",6.7,[101],23107.0,1995.0,113228
3,"[u'Argentina:16', u'Australia:M', u'Canada:14A...",[us],"[Comedy, Drama, Romance]",movie,[en],"[u""Based on Terry McMillan's novel, this film ...",5.9,[124],9034.0,1995.0,114885
4,"[u'Argentina:Atp', u'Australia:G', u'Finland:S...",[us],"[Comedy, Family, Romance]",movie,[en],[u'George Banks must deal not only with the pr...,6.0,[106],32568.0,1995.0,113041


(43697, 48910)